<a href="https://colab.research.google.com/github/SHEHAN-120/Churn-Prediction-using-CatBoost-Classifier/blob/main/Churn_modeling_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Data Preprocessing

In [1]:
import pandas as pd
dataset = pd.read_csv('churn_modelling.csv')

In [2]:
dataset.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  object 
 4   Gender           10000 non-null  object 
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  int64  
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 1015.8+ KB


In [4]:
dataset.drop(['CustomerId', 'Surname'], axis = 1, inplace = True)

In [5]:
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
dataset['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [7]:
geography_dummies = pd.get_dummies(dataset['Geography'], drop_first = True)

In [8]:
geography_dummies

,Germany,Spain
0,False,False
1,False,True
2,False,False
3,False,False
4,False,True
...,...,...
9995,False,False
9996,False,False
9997,False,False
9998,True,False


In [9]:
dataset = pd.concat([geography_dummies, dataset], axis = 1)

In [10]:
dataset.head()

,Germany,Spain,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,False,False,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,False,True,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,False,False,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,False,False,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,False,True,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
dataset.drop(['Geography'], axis = 1, inplace = True)

In [12]:
dataset.head()

,Germany,Spain,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,False,False,619,Female,42,2,0.00,1,1,1,101348.88,1
1,False,True,608,Female,41,1,83807.86,1,0,1,112542.58,0
2,False,False,502,Female,42,8,159660.80,3,1,0,113931.57,1
3,False,False,699,Female,39,1,0.00,2,0,0,93826.63,0
4,False,True,850,Female,43,2,125510.82,1,1,1,79084.10,0


In [13]:
dataset['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [14]:
dataset['Gender'] = dataset['Gender'].apply(lambda x: 0 if x == 'Female' else 1)

In [15]:
dataset.head()

,Germany,Spain,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,False,False,619,0,42,2,0.00,1,1,1,101348.88,1
1,False,True,608,0,41,1,83807.86,1,0,1,112542.58,0
2,False,False,502,0,42,8,159660.80,3,1,0,113931.57,1
3,False,False,699,0,39,1,0.00,2,0,0,93826.63,0
4,False,True,850,0,43,2,125510.82,1,1,1,79084.10,0


In [16]:
X = dataset.iloc[:, :-1].values

In [17]:
y = dataset.iloc[:, -1].values

In [18]:
X

array([[False, False, 619, ..., 1, 1, 101348.88],
       [False, True, 608, ..., 0, 1, 112542.58],
       [False, False, 502, ..., 1, 0, 113931.57],
       ...,
       [False, False, 709, ..., 0, 1, 42085.58],
       [True, False, 772, ..., 1, 0, 92888.52],
       [False, False, 792, ..., 1, 0, 38190.78]], dtype=object)

In [19]:
y

array([1, 0, 1, ..., 1, 1, 0])

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Building and training the model

In [21]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.9 MB/s eta 0:00:00


In [22]:
import catboost as cb
model = cb.CatBoostClassifier()

In [23]:
model.fit(X_train, y_train)

Learning rate set to 0.025035
0:	learn: 0.6715999	total: 52.1ms	remaining: 52.1s
1:	learn: 0.6523077	total: 61.9ms	remaining: 30.9s
2:	learn: 0.6343125	total: 69.7ms	remaining: 23.2s
3:	learn: 0.6177234	total: 75.4ms	remaining: 18.8s
4:	learn: 0.6020753	total: 81.6ms	remaining: 16.2s
5:	learn: 0.5879791	total: 89.1ms	remaining: 14.8s
6:	learn: 0.5754312	total: 92.1ms	remaining: 13.1s
7:	learn: 0.5639214	total: 101ms	remaining: 12.5s
8:	learn: 0.5531841	total: 110ms	remaining: 12.1s
9:	learn: 0.5433279	total: 119ms	remaining: 11.8s
10:	learn: 0.5316836	total: 131ms	remaining: 11.8s
11:	learn: 0.5232186	total: 140ms	remaining: 11.5s
12:	learn: 0.5128050	total: 153ms	remaining: 11.6s
13:	learn: 0.5025235	total: 167ms	remaining: 11.8s
14:	learn: 0.4939152	total: 176ms	remaining: 11.6s
15:	learn: 0.4847706	total: 188ms	remaining: 11.6s
16:	learn: 0.4784245	total: 200ms	remaining: 11.6s
17:	learn: 0.4709868	total: 208ms	remaining: 11.3s
18:	learn: 0.4643611	total: 218ms	remaining: 11.2s
19:	

In [24]:
y_pred = model.predict(X_test)

##  Evaluating the model

### Making the Confusion Matrix

In [25]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[1514,   81],
       [ 189,  216]])

### Accuracy

In [26]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.865